# Matrix Factorization with NN (Problem)

* Do matrix factorization by NN and to obtain weights as vectors that encode each column and row entries.
* In this example table, we want to learn a vector representation for cat, dog, run, jump, etc., such that the inner products of (cat, run) = 0.85, (cat, jump) = 0.92, ... and so on.   

|  -  | run  | jump | laugh | sleep |
|-----|------|------|-------|-------|
| cat | 0.85 | 0.92 | 0.95  | 1.37  | 
| dog | 0.62 | 0.77 | 0.78  | 1.14  |
  
* Question inspired by the lecture video [Deep Learning for Language Model](https://youtu.be/Jvigef51rqk)


In [1]:
import warnings
warnings.simplefilter('ignore', category=FutureWarning)

import numpy as np
import tensorflow as tf

print(tf.__version__)
np.set_printoptions(precision=4, suppress=True)

1.10.0


## Matrix M
Here M is generated as the matrix multiplication of M1 and M2, which are not known beforehand. They naturally form a possible answer tot he factorization problem. But in general we will not get them back as our result since the answer to the factorization is not unique. 

In [2]:
# Dimension of M
num_row = 2
num_col = 4

# The size of the hidden layer correspondes to the intended size of vectors to be learned  
hidden_size = 3

# Training epoch
num_epoch = 20000

In [3]:
np.random.seed(0)
M1 = np.random.random([num_row, hidden_size])
M2 = np.random.random([hidden_size, num_col]) 
M = np.matmul(M1, M2)
print('M:\n', M)

M:
 [[0.8492 0.9202 0.9473 1.3743]
 [0.6197 0.7663 0.7788 1.1388]]


## Construct newtork and train

* Each row of $W_1$ represents a vector (cat, dog)
* Each column of $W_2$ represents a vector (run, jump, laugh, sleep)
* Without having any activation functions and bias term, this is really just matrix multiplication (as intended) and does not qualify a real neural network.


In [4]:

x = tf.placeholder(tf.float32, [None, num_row])
y = tf.placeholder(tf.float32, [None, num_col])

# Matrix multiplication without bias terms
W1 = tf.Variable(tf.random_normal([num_row, hidden_size]))
l1 = tf.matmul(x, W1)
W2 = tf.Variable(tf.random_normal([hidden_size, num_col]))
pred = tf.matmul(l1, W2)
loss = tf.reduce_mean(tf.square(pred - y)) 

optimize_step = tf.train.AdamOptimizer().minimize(loss)

x_data = np.eye(M.shape[0])
y_data = M

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    out_pred, out_loss = sess.run([pred, loss], feed_dict={x:x_data, y:y_data})
    print('initial loss: ', out_loss)
    
    for i in range(num_epoch):
        sess.run([optimize_step, loss], feed_dict={x:x_data, y:y_data})
        if i % 1000 == 0:
            print(i, ' loss ', sess.run(loss, feed_dict={x:x_data, y:y_data}))
    W1, W2 = sess.run([W1, W2])
    print('W1\n', W1)
    print('W2\n', W2)

initial loss:  9.533157
0  loss  9.508075
1000  loss  0.7905307
2000  loss  0.075779274
3000  loss  0.018555598
4000  loss  0.0046997042
5000  loss  0.0008214359
6000  loss  5.5570927e-05
7000  loss  6.3316475e-07
8000  loss  3.6362646e-10
9000  loss  8.686385e-13
10000  loss  1.6342483e-13
11000  loss  8.482104e-14
12000  loss  1.8651747e-14
13000  loss  5.7731597e-15
14000  loss  3.5527137e-15
15000  loss  3.1086245e-15
16000  loss  1.731948e-14
17000  loss  2.5228708e-12
18000  loss  5.216272e-12
19000  loss  2.5728308e-11
W1
 [[ 0.9582  1.2214 -1.173 ]
 [ 0.2596  0.5451  0.2061]]
W2
 [[-1.3521 -0.9959 -0.0032  1.2053]
 [ 1.7738  1.7825  1.246   1.1379]
 [ 0.0186  0.258   0.4872  0.9978]]


## MSE of reproduced matrix 

In [5]:
print('M (data matrix)\n', M)
M_reproduced = np.matmul(W1, W2)
print('W1W2 (reproduced matrix)\n', M_reproduced)
print('Residual:\n', M - M_reproduced)
rmse_M = np.sqrt(np.mean(np.square((M - M_reproduced))))
print('mean error: {}'.format(rmse_M))

M (data matrix)
 [[0.8492 0.9202 0.9473 1.3743]
 [0.6197 0.7663 0.7788 1.1388]]
W1W2 (reproduced matrix)
 [[0.8492 0.9202 0.9473 1.3743]
 [0.6197 0.7662 0.7788 1.1388]]
Residual:
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
mean error: 2.939346621565998e-05


## Summary

1. The method appears to work, at least on the current example. It finds the factorized matrices $W_1$ and $W_2$ that give $W_1W_2 \simeq M$. The rmse error can be lower than 1e-5.


## Note

1. Does this work on a larger matrix M?
2. Does this work for any number of hidden layers?  

See [1_Experiments.ipynb](1_Experiments.ipynb) 
